# Probability Calibration in KG Embedding
This experiemnt is to investigate which calibration technique is the most suitable one given a dataset and a KG Embedding model.

Within this experiment, we are going to see the performance of 4 typical calibration techniques for 4 KGE models in 3 datasets:
- calibration techniques:
  - Platt Scaling
  - Isotonic Regression
  - Histogram BInning
  - Beta Calibration
- KG Embedding models
  - TransE
  - ComplEx
  - DistMult
  - HoLE
- Datasets
  - FB13k
  - Win11
  - Yago39

In [1]:
import sys
# enable importing the modules from probcalkge
sys.path.append('../')
sys.path.append('../probcalkge')

In [2]:
import importlib
from pprint import pprint
import numpy as np
import pandas as pd

In [4]:
from ampligraph.latent_features import RandomBaseline, TransE
import probcalkge
importlib.reload(probcalkge)
from probcalkge import Experiment
from probcalkge import get_calibrators
from probcalkge import get_datasets, get_fb13, get_wn11, get_kgemodels, get_yago39
from probcalkge import brier_score, negative_log_loss, ks_error

In [6]:
ds = get_datasets()
cals = get_calibrators()
kges = get_kgemodels()


WARNING - All triples will be processed in the same batch (batches_count=1). When processing large graphs it is recommended to batch the input knowledge graph instead.


In [8]:
exp = Experiment(
    cals=[cals.uncal, cals.platt, cals.isot, cals.histbin, cals.beta], 
    datasets=[ds.fb13, ds.wn18, ds.yago39, ds.dp50, ds.nations, ds.kinship, ds.umls], 
    kges=[kges.transE, kges.complEx, kges.distMult, kges.hoLE], 
    metrics=[brier_score, negative_log_loss, ks_error]
    )
exp.load_trained_kges('../saved_models/')
# exp.train_kges()

Loaded models:
{'DBpedia50': {'ComplEx': <ampligraph.latent_features.models.ComplEx.ComplEx object at 0x000001FE63181B08>,
               'DistMult': <ampligraph.latent_features.models.DistMult.DistMult object at 0x000001FE67EB2048>,
               'HolE': <ampligraph.latent_features.models.HolE.HolE object at 0x000001FE67EFA048>,
               'TransE': <ampligraph.latent_features.models.TransE.TransE object at 0x000001FE68426048>},
 'FB13k': {'ComplEx': <ampligraph.latent_features.models.ComplEx.ComplEx object at 0x000001FE678E9048>,
           'DistMult': <ampligraph.latent_features.models.DistMult.DistMult object at 0x000001FE67831BC8>,
           'HolE': <ampligraph.latent_features.models.HolE.HolE object at 0x000001FE67EB2248>,
           'TransE': <ampligraph.latent_features.models.TransE.TransE object at 0x000001FE67EFA248>},
 'Kinship': {'ComplEx': <ampligraph.latent_features.models.ComplEx.ComplEx object at 0x000001FE68426288>,
             'DistMult': <ampligraph.latent_fea

In [9]:
exp_res = exp.run_with_trained_kges()

{'FB13k': {'TransE':                    UncalCalibrator  PlattCalibrator  IsotonicCalibrator  \
brier_score               0.242014         0.212112            0.206325   
negative_log_loss         0.676226         0.616217            0.600623   
ks_error                  0.098866         0.027720            0.004463   

                   HistogramBinningCalibrator  BetaCalibrator  
brier_score                          0.215394        0.209044  
negative_log_loss                    0.618863        0.609719  
ks_error                             0.004041        0.016290  , 'ComplEx':                    UncalCalibrator  PlattCalibrator  IsotonicCalibrator  \
brier_score               0.410170         0.222095            0.208440   
negative_log_loss         2.252936         0.633720            0.602848   
ks_error                  0.421845         0.049897            0.002977   

                   HistogramBinningCalibrator  BetaCalibrator  
brier_score                          0.225157

In [10]:
exp_res.to_frame()

ExpRes
dataset kge    cal                        metric                     
FB13k   TransE UncalCalibrator            brier_score        0.242014
                                          negative_log_loss  0.676226
                                          ks_error           0.098866
               PlattCalibrator            brier_score        0.212112
                                          negative_log_loss  0.616217
...                                                               ...
UMLS    HolE   HistogramBinningCalibrator negative_log_loss  0.321492
                                          ks_error           0.016966
               BetaCalibrator             brier_score        0.096088
                                          negative_log_loss  0.317385
                                          ks_error           0.008779

[420 rows x 1 columns]